In [ ]:
# data and its procession
import numpy as np
import time
import matplotlib.pylab as plt
import matplotlib.animation as animation
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

data = np.random.rand(1000,100)
def single_run(data,queue):
    import numpy as np
    import time
    for i_loop in np.arange(data.shape[0]):
        time.sleep(0.02)
    queue.put(data)

In [ ]:
# multi core process function
def multi_process(data,single_run,n_job):
    import multiprocessing as mp
    from pathos.multiprocessing import ProcessingPool as Pool
    # set right cpu number
    n_cpu = mp.cpu_count()
    if (n_job>0) and (n_job<=n_cpu):
        n_job = int(n_job)
    else:
        n_job = n_cpu-1
    # chunk data
    n_vox = data.shape[0]
    i_chunk = int(np.floor(n_vox/n_job))
    index_chunk = np.arange(n_job)*i_chunk
    procs = []
    manager = mp.Manager()
    queue = manager.Queue()
    for i_job in np.arange(n_job):
        if i_job==n_job-1:
            data_job = data[index_chunk[i_job]:,:]
        else:
            data_job = data[index_chunk[i_job]:index_chunk[i_job+1],:]
        # multi process
        proc = mp.Process(target=single_run,args=(data_job,queue))
        procs.append(proc)
    for proc in procs:
        proc.start()
    for proc in procs:
        proc.join()
    results = np.zeros(data.shape)
    for i_job in np.arange(n_job):
        res = queue.get()
        if i_job==n_job-1:
            results[index_chunk[i_job]:,:]=res
        else:
            results[index_chunk[i_job]:index_chunk[i_job+1],:]=res
    return results
        

In [ ]:
# multi core processing and time display
n_core_array = np.arange(1,8,1)
time_array = np.zeros(n_core_array.shape)
for i_run,i_core in enumerate(n_core_array):
    time_begin = time.time()
    results = multi_process(data,single_run,i_core)
    time_end = time.time()
    total_time2 = time_end-time_begin
    time_array[i_run] = total_time2

print('time used: ',time_array)

In [ ]:
import matplotlib.pylab as plt
fig,ax = plt.subplots()
ax.plot(n_core_array,time_array,label='test',linestyle='--',marker='s',markersize=8,color='royalblue')
plt.xlabel(r'n cpu')
plt.ylabel(r'time [s]')
plt.legend()
plt.title(r'multi cores test')
plt.savefig('./test/multi_core_test.png', dpi=300,
         bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
#initilize w
# w = np.zeros((x.shape[1],1))

# create figures
fig, ax = plt.subplots()
fig.subplots_adjust(left=0.09, bottom=0.09, right=0.99, top=0.99,wspace=None, hspace=None)#, 

ims = []

#Optimization loop
for i_w,i_core in enumerate(n_core_array):
    # show results
    im, = ax.plot(n_core_array[0:i_w],time_array[0:i_w],linestyle='--',color='royalblue',marker='s',markersize=8)
    title= ax.text(0.78,0.88,r'number of cpu='+str(n_core_array[i_w])+r', time used='+str(np.round(time_array[i_w],2))+'s', 
                    size=plt.rcParams["axes.titlesize"],
                    ha="center", transform=ax.transAxes, )
    plt.xlabel(r'number of cpu used')
    plt.ylabel(r'time used [s]')
    ims.append([im,title])
ax.plot(n_core_array[0],time_array[0],linestyle='--',color='royalblue',marker='s',markersize=8,label='test')
plt.legend(loc='upper right')
ani = animation.ArtistAnimation(fig, ims, interval=800, blit=False)
ani.save("./image/multi_core_test.gif",writer='pillow',dpi=300)
plt.show()
